## Installing the Pinecone and LangChain libraries required for the Chatbot

In [49]:
# !pip install \
#   langchain_community \
#   langchain_pinecone \
#   langchain_openai \
#   unstructured \
#   langchain-text-splitters \
#   pinecone-text

## Importing the Langchain and Pinecone Modules from the libraries

In [50]:

from langchain_pinecone import PineconeVectorStore
from langchain_community.retrievers import (
    PineconeHybridSearchRetriever)
from pinecone.grpc import PineconeGRPC as Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import TextLoader
from langchain_core.runnables import Runnable
from pinecone import ServerlessSpec
import pandas as pd
import itertools
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
from dotenv import load_dotenv
import pinecone
import uuid
import os
import glob
import hashlib
from tqdm.autonotebook import tqdm

In [51]:
# List all files in the 'data' directory
#print("Files in 'data' directory:", os.listdir('data'))

In [52]:
directory = 'data'

def load_docs(directory):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    return docs

docs = load_docs(directory)
print(len(docs))

3


In [53]:
#print(f"Loaded {len(docs)} documents.")


## Natural Language ToolKit

In [54]:
#  import nltk
#  nltk.download('punkt')

## API Keys Verifications

In [55]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Fetch API keys from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')

# Set the environment variables
if openai_api_key:
    os.environ['OPENAI_API_KEY'] = openai_api_key
if pinecone_api_key:
    os.environ['PINECONE_API_KEY'] = pinecone_api_key

#Verify that the keys are loaded
#print(f"OpenAI API Key: {os.environ.get('OPENAI_API_KEY')}")
print(f"Pinecone API Key: {os.environ.get('PINECONE_API_KEY')}")

Pinecone API Key: 3e7dee19-83b9-46f2-88b4-5a549d18f705


In [56]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [57]:
import time

index_name = "test-2"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [59]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]="HUGGINGFACEHUB_API_TOKEN"
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",  #response time is 9s  #infloat/e5-base-V2 has 3.53sec response time.
)
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x329a37560>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x32424fda0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [60]:
from langchain_pinecone import PineconeVectorStore

vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

In [61]:
# from uuid import uuid4

# from langchain_core.documents import Document

# document_1 = Document(
#     page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
#     metadata={"source": "tweet"},
# )

# document_2 = Document(
#     page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
#     metadata={"source": "news"},
# )

# document_3 = Document(
#     page_content="Building an exciting new project with LangChain - come check it out!",
#     metadata={"source": "tweet"},
# )

# document_4 = Document(
#     page_content="Robbers broke into the city bank and stole $1 million in cash.",
#     metadata={"source": "news"},
# )

# document_5 = Document(
#     page_content="Wow! That was an amazing movie. I can't wait to see it again.",
#     metadata={"source": "tweet"},
# )

# document_6 = Document(
#     page_content="Is the new iPhone worth the price? Read this review to find out.",
#     metadata={"source": "website"},
# )

# document_7 = Document(
#     page_content="The top 10 soccer players in the world right now.",
#     metadata={"source": "website"},
# )

# document_8 = Document(
#     page_content="LangGraph is the best framework for building stateful, agentic applications!",
#     metadata={"source": "tweet"},
# )

# document_9 = Document(
#     page_content="The stock market is down 500 points today due to fears of a recession.",
#     metadata={"source": "news"},
# )

# document_10 = Document(
#     page_content="I have a bad feeling I am going to get deleted :(",
#     metadata={"source": "tweet"},
# )

# documents = [
#     document_1,
#     document_2,
#     document_3,
#     document_4,
#     document_5,
#     document_6,
#     document_7,
#     document_8,
#     document_9,
#     document_10,
# ]
# uuids = [str(uuid4()) for _ in range(len(documents))]

# vector_store.add_documents(documents=documents, ids=uuids)

In [69]:
import os
from uuid import uuid4
from langchain_core.documents import Document
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Pinecone  # Adjust according to your vector store setup


import time

index_name = "test-2"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

# Directory containing your documents (e.g., text files, PDFs)
directory = 'data'

# Function to load documents from the specified directory
def load_docs(directory):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    return docs

# Load documents from the directory
docs = load_docs(directory)
print(f"Loaded {len(docs)} documents from '{directory}'.")

# Prepare texts and metadata for each document
texts = [doc.page_content for doc in docs]  # Assuming docs have a page_content attribute
metadata = [{"source": os.path.basename(doc.metadata['source']), "text": doc.page_content} for doc in docs]

# Generate unique IDs for each document using uuid4()
ids = [str(uuid4()) for _ in range(len(texts))]

# Print generated IDs along with document content and metadata
print("Generated Document IDs and their corresponding texts:")
for doc_id, text, meta in zip(ids, texts, metadata):
    print(f"ID: {doc_id}, Source: {meta['source']}, Text: {text[:30]}...")  # Print first 30 characters of text

# Initialize your vector store (e.g., Pinecone)
vector_store = Pinecone(index=index, embedding=embeddings, text_key=text)

# Add texts to Pinecone with generated IDs and metadata
#vector_store.add_documents(documents=texts, ids=ids, metadatas=metadata)

print("Documents added to Pinecone successfully!")

Loaded 3 documents from 'data'.
Generated Document IDs and their corresponding texts:
ID: 3bde953d-119a-4cb1-8123-e90f58fc65f1, Source: Document.pdf, Text: Documentation on Talk to your ...
ID: 5914d084-7bbc-451e-898d-036d635846aa, Source: Clients Module ✓.docx, Text: support@vitafyhealth.com | 1-8...
ID: f5d01dc8-fcec-4c97-8e0a-53b0a0bce0db, Source: Retrieval Augmented Generation (RAG) for Everyone.docx, Text: Retrieval Augmented Generation...
Documents added to Pinecone successfully!


In [ ]:
# Example: Deleting a vector from Pinecone using its ID
#vector_store.index.delete(ids=['5b85a555-54c1-4a32-b15f-a0c81a8709b8'])

## OpenAI Embeddings Model (Dense Vectors)

In [13]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]="HUGGINGFACEHUB_API_TOKEN"
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",  #response time is 9s  #infloat/e5-base-V2 has 3.53sec response time.
)
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x326e2c6b0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x326df8cb0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

## Pinecone Langchain Store

In [48]:
#pc.list_indexes()


## Upload & Genereate Docs Vector ID

In [49]:
# index_name = "test-2"
# index = pinecone.Index(index_name, host='test-2-5vwf04k.svc.aped-4627-b74a.pinecone.io')

# # Load documents from the specified directory
# directory = 'data'  # Change this path to your actual directory containing the documents

# def load_docs(directory):
#     loader = DirectoryLoader(directory)
#     docs = loader.load()
#     return docs

# docs = load_docs(directory)

# # Initialize the embeddings model
# embeddings = OpenAIEmbeddings(
#     model="text-embedding-ada-002",  # You can adjust to another model if needed
# )

# # Split the documents for embeddings (if necessary)
# doc_texts = [doc.page_content for doc in docs]
# doc_ids = [f"doc_{i}" for i in range(len(docs))]  # Generate unique IDs for each document

# # Generate embeddings for the documents
# doc_embeddings = embeddings.embed_documents(doc_texts)

# # Prepare the data for upsert (list of tuples with (ID, embedding))
# #upsert_data = list(zip(doc_ids, doc_embeddings))
# # Prepare the data for upsert with correct format
# upsert_data = [
#     {"id": doc_id, "values": embedding} 
#     for doc_id, embedding in zip(doc_ids, doc_embeddings)
# ]

# # Upsert documents in batches
# batch_size = 100  

# for i in range(0, len(upsert_data), batch_size):
#     batch = upsert_data[i:i + batch_size]
#     # Upsert the batch to Pinecone
#     index.upsert(vectors=batch)
#     print(f"Upserted batch {i // batch_size + 1} of {len(upsert_data) // batch_size + 1}")

# print(f"Upserted all {len(upsert_data)} documents into Pinecone index '{index_name}'.")

# # Fetch and verify the stored vectors (Optional)
# for doc_id, doc in zip(doc_ids, docs):
#     #
#     result = index.fetch(ids=[doc_id])
#     if result and 'vectors' in result:
#         vector = result['vectors'].get(doc_id, {}).get('values', [])
#         filename = doc.metadata.get('filename', 'unknown')
#         print(f"Filename: {filename} - Document ID: {doc_id} - Vector: {vector[:1]}... (truncated)")

ForbiddenException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'Date': 'Thu, 19 Sep 2024 04:55:23 GMT', 'Content-Type': 'text/plain', 'Content-Length': '9', 'Connection': 'keep-alive', 'x-pinecone-auth-rejected-reason': 'Wrong API key', 'www-authenticate': 'Wrong API key', 'server': 'envoy'})
HTTP response body: Forbidden


## Split Docs

In [662]:

chunk_size = 1000  
chunk_overlap = 200  

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

split_docs = text_splitter.split_documents(docs)

## Pinecone Vector Store

In [663]:
index_name = "test-2"
vectorstore = PineconeVectorStore.from_documents(split_docs, embeddings, index_name=index_name)
vectorstore

## query = Questions

In [664]:
keyword = """ What is RAG??"""

## Retriever

In [665]:
retriever = vectorstore.as_retriever(search_kwargs = {"k":5})
retriever.get_relevant_documents(keyword)
retriever

VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x39c69b4a0>, search_kwargs={'k': 5})

## Importing Chat Model as GPT-4o

In [666]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.7,
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),  # Set to retrieve only 1 document
    return_source_documents=True  # This will return source documents in the response

)

qa.invoke(keyword)

{'query': ' How to create a membership??',
 'result': "To create a membership, follow these steps:\n\n1. Go to the **Settings** > **Membership Offerings** tab.\n\n2. Click on **+Add** on the top right side of the page to add a Membership type.\n\n3. Choose **Category**: Select either **Individual & Family** OR **Company & Group**.\n\n4. Choose **Age range** and **Quantity** (minimum and maximum number of members if any) by typing manually or using the arrow buttons on the right-hand side.\n\n5. Set the **Price**.\n\n6. Set the **Billing Interval** from the dropdown options: **Bi-yearly, Monthly, One Time, Quarterly, Yearly**.\n\n7. Set the **Billing Type** from the dropdown:\n   - **By Family**\n   - **Group**: All members in a group pay together.\n   - **Individual**: Each individual in the group is charged separately.\n\n8. Add a **Registration Fee**: This is a one-time registration fee.\n\n9. Choose **Visibility** from the dropdown:\n   - **Public**: Shown on the public site and can

In [667]:
# # Post-process the output
# response = qa.invoke(query)
# # result = response.get('result', 'No result found')
# source_documents = response.get('source_documents', 'No source documents available')
# source_info = response['source_documents']  
# print(f"Response: {response['result']} (Source: {source_info})")

## Keyword and Response (with Pinecone and without Pinecone)

In [668]:
# keyword = """ According to the Constitution of the Kingdom of Nepal 1990, there is a
# provision for electing how members to the House of
# Representatives."""

# # Send each keyword to the LLM twice, first with relevant knowledge from Pincone 
# # and then without any additional knowledge.
# print("Response \n")
# print("Chat with Pinecone:")
# print(qa.invoke(keyword).get("result"))
# #print("\nChat with GPT-4o:")
# #print(llm.invoke(keyword).content)
# # Combine the two responses for clarity
# #print("\nCombined Response (Pinecone + GPT-4o):")
# #combined_response = f"Pinecone Response: {"Chat with Pinecone:"}\nGPT-4o Response: {"\nChat with GPT-4o:"}"
# #print(combined_response)


## Prompt Template

In [669]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retrieved_docs = retriever.invoke(keyword)
#print(format_docs(retrieved_docs))

In [670]:
template = """You are an expert LLM assistant specialized in answering questions based solely on the information provided in the uploaded documents (PDF, DOCX, or TXT formats). Use only the information from the documents to respond accurately and clearly to each question.

Guidelines:
1. Provide concise and informative answers.
2. If the answer is not found in the uploaded documents, state, "The answer is not available in the provided documents."
3. Avoid using outside knowledge or assumptions. Stick strictly to the content in the documents.
4. Maintain a professional and helpful tone.
5. Answer for normal conversation question like "Hi", "Hey", "Hello", "How are you", and many others questions with answer "Hello, How can I assist you?".
6. If question is on "summarize" or "summerization", then summarize the documents and (1/4)th the size of documents.

Question: {question}

Context: {context}

Answer:
"""
prompt = template.format(question = keyword, context =  format_docs(retrieved_docs))


## Chains

In [671]:
llm = OpenAI(api_key=openai_api_key)

custom_rag_template = PromptTemplate.from_template(template)

# Create the parallel chain
My_rag_chain = RunnableParallel(
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
) | custom_rag_template | llm | StrOutputParser()


## My chain : Retriever(Pinecone) | custom_rag_template(prompt) | llm | StrOutputParser()

## keyword and Response (with Pinecone and without Pinecone)

In [672]:
keyword = """ How to create a membership?"""

print("Chat with your Documents:")
print(My_rag_chain.invoke(keyword))
#print("\nChat with GPT-4o:")
#print(llm.invoke(keyword).content)
# Combine the two responses for clarity
#print("\nCombined Response (Pinecone + GPT-4o):")
#combined_response = f"Pinecone Response: {"Chat with Pinecone:"}\nGPT-4o Response: {"\nChat with GPT-4o:"}"
#print(combined_response)

Chat with your Documents:

To create a membership, go to Settings > Membership Offerings tab and click on +Add. Then, choose the category (Individual & Family or Company & Group), set the age range and quantity, and set the price. Next, select the billing interval and type, and add a registration fee if desired. Choose the visibility (public or private) and status (active or inactive), and set the start and end date. Repeat these steps for each membership type you want to offer.


## Gradio

In [673]:
# import gradio as gr

# def llm_response(keyword, memory = None):
#     return My_rag_chain.invoke(keyword)

# rag_demo = gr.ChatInterface(
#     llm_response, 
#     title="RAG demo",
#     chatbot=gr.Chatbot(height=300),
#     textbox=gr.Textbox(placeholder="Enter keyword here", scale=5),
#     examples=["Hello"],
#     retry_btn=gr.Button("Retry"),
#     clear_btn=gr.Button("Clear"),
#     undo_btn=gr.Button("Undo"),
#     submit_btn=gr.Button("Submit")
# )

In [674]:
#rag_demo.launch(share=False)

In [675]:
# # Define 25 tenants, each with 50 users
# tenants = {f"tenant_{i}": [f"user_{i}_{j}" for j in range(50)] for i in range(1, 26)}

# # Print out the tenants and their users
# for tenant_id, users in tenants.items():
#     print(f"Tenant ID: {tenant_id}")
#     for user in users:
#         print(f"  User: {user}")


## Folder, Upload Docs, Delete Docs, Isolation keyword

In [676]:
# # User Folders
# def create_user_folders(base_path, tenants):
#     """Create folders for each tenant and their users."""
#     for tenant_id, users in tenants.items():
#         tenant_path = os.path.join(base_path, tenant_id)
#         os.makedirs(tenant_path, exist_ok=True)
#         for user in users:
#             user_path = os.path.join(tenant_path, user)
#             os.makedirs(user_path, exist_ok=True)
#     print("User folders created successfully.")

# #Uploading File
# def upload_file(base_path, tenant_id, user_id, file_path):
#     """Upload a file and store its embedding with metadata for tenant filtering."""
#     tenant_folder = os.path.join(base_path, tenant_id)
#     user_folder = os.path.join(tenant_folder, user_id)
    
#     if not os.path.exists(user_folder):
#         raise ValueError(f"User folder {user_folder} does not exist")
    
#     destination = os.path.join(user_folder, os.path.basename(file_path))
    
#     if os.path.exists(destination):
#         raise FileExistsError(f"File {destination} already exists")
    
#     try:
#         with open(file_path, 'rb') as fsrc, open(destination, 'wb') as fdst:
#             fdst.write(fsrc.read())
        
#         # Create embeddings and store in Pinecone with tenant metadata
#         embedding = embeddings.embed_documents(file_path)
#         metadata = {"tenant_id": tenant_id, "user_id": user_id}
#         vectorstore.add(vectors=[embedding], ids=[os.path.basename(file_path)], metadata=metadata)
#         print(f"File {file_path} uploaded to {destination}")
#     except Exception as e:
#         print(f"An error occurred while uploading the file: {e}")
    
        
        
# #  Delete Documents with pinecone embeddings    
# def delete_file(base_path, tenant_id, user_id, vector_ID):
#     """Delete a file from the specified tenant and user folder, and remove its embedding from Pinecone."""
#     file_path = os.path.join(base_path, tenant_id, user_id, vector_ID)
    
#     if os.path.exists(file_path):
#         try:
#             # Remove the file from the filesystem
#             os.remove(file_path)
#             print(f"File {file_path} deleted successfully.")
            
#             # Remove the corresponding embedding from Pinecone using metadata filter
#             meta_data = {"tenant_id": tenant_id, "user_id": user_id}  # Define metadata filter
#             vectorstore.delete(filter=meta_data)  # Delete vectors matching the metadata filter # base_path/tenant_id/user_id/vector_ID ~ vector_ID=vector_id
#             print(f"Embedding for {vector_ID} deleted from Pinecone along with its metadata.")
            
#         except Exception as e:
#             print(f"An error occurred while deleting the file: {e}")
#     else:
#         print(f"File {file_path} not found.")       
    


# ##### Creating tenant and user for document storage  ######
# #####                                  ######
# tenants = {
#     'tenant_1': ['user_1_0', 'user_1_1','user_1_2','user_1_3','user_1_4',],
#     'tenant_2': ['user_2_0', 'user_2_1','user_2_2','user_2_3','user_2_4',],
#     'tenant_3': ['user_3_0', 'user_3_1','user_3_2','user_3_3','user_3_4',],
#     'tenant_4': ['user_4_0', 'user_4_1','user_4_2','user_4_3','user_4_4',],
#     'tenant_5': ['user_5_0', 'user_5_1','user_5_2','user_5_3','user_5_4',],
#     # Add more tenants and users as needed
# }

# # Base directory where user folders will be created
# base_path = 'file_storage'

# # Create folders
# create_user_folders(base_path, tenants)

# # Example of uploading a file
# upload_file(base_path, 'tenant_1', 'user_1_1', '/Users/user/Downloads/Chatbot with OpenAI embedding models, GPT, Langchain and Pinecone (1).docx')

# # Example of deleting a file
# #delete_file(base_path, 'tenant_1', 'user_1_1', 'Retrieval Augmented Generation (RAG) for Everyone.docx')


In [677]:
#Get an Pinecone index endpoint = https://ujjwaln-rn229jx.svc.aped-4627-b74a.pinecone.io
#Get an openai endpoint = https://api.openai.com/v1/chat/completions (POST) - Creates a model response for the given chat conversation.





In [678]:
'source_documents': [Document(id='c1c6bfb8-6672-4ee3-9089-e83b73481c86', metadata={'source': 'data/Settings Module.docx'}, page_content='Go to Settings >Membership Offerings tab.\n\nClick on +Add on the top right side of the page to add a Membership type:\n\nChoose Category: Individual & Family OR Company & Group\n\nChoose Age range and Quantity (min and max number of members if any) can be set by typing manually or using the arrow buttons on the right hand side.\n\nSet Price.\n\nSet Billing Interval from dropdown: Bi-yearly, Monthly, One Time, Quarterly, Yearly.\n\nSet Billing Type from dropdown:\n\nBy Family\n\nGroup: all members in a group pay together\n\nIndividual: each individual in group charged separately\n\nAdd Registration Fee: One time registration fee\n\nChoose Visibility from dropdown:\n\nPublic: Shown on the public site and can be chosen as an option by anyone.\n\nPrivate: Can only be chosen as an option by admin.\n\nChoose Status:\n\nActive:  Can be chosen as an option\n\nInactive: Cannot be chosen as an option\n\nAdd Start and End Date.')]}


SyntaxError: unmatched '}' (364757047.py, line 1)